<a href="https://colab.research.google.com/github/leideng/AI-primer/blob/main/pytorch/block_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import torch
times = 1
block_size = 128
batch_size = 5*times
max_num_blocks_per_req = 10*times
num_blocks = 100*times
top_k = max_num_blocks_per_req // 2

print(f"block_size={block_size}")
print(f"batch_size={batch_size}")
print(f"max_num_blocks_per_req={max_num_blocks_per_req}")
print(f"num_blocks={num_blocks}")
print(f"top_k={top_k}")

block_table = torch.randperm(num_blocks, device="cuda").view(batch_size, -1)
block_table[:,max_num_blocks_per_req:] = -1
print(f"block_table=\n{block_table}")

block_size=128
batch_size=5
max_num_blocks_per_req=10
num_blocks=100
top_k=5


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [51]:
#torch.randint(3, 5, (3,))
seq_lens = torch.randint(low=1,high=max_num_blocks_per_req*block_size, size=(batch_size,))
print(f"seq_lens={seq_lens}")

num_valid_blocks = (seq_lens + block_size -1) // block_size
print(f"num_valid_blocks={num_valid_blocks}")

for i in range(batch_size):
  block_table[i,num_valid_blocks[i]:-1] = -1
print(f"block_table=\n{block_table}")

seq_lens=tensor([1161,  198,  467,  823,  511])
num_valid_blocks=tensor([10,  2,  4,  7,  4])
block_table=
tensor([[59, 69, 65, 76, 64, 57,  2, 60, 81, 79, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [96, 83, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [47, 38, 80,  9, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [74,  4, 70, 78, 53,  0, 12, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1],
        [10, 19, 82,  7, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
         -1, -1]], device='cuda:0')


In [61]:
topk_block_indices = block_table[:,:top_k]

for i in range(batch_size):
  perm = torch.randperm(top_k)
  sorted_perm = block_table[i,perm]
  print(f"i={i}, perm={perm}, sorted_perm={sorted_perm}")
  #topk_block_indices[i,:] = block_table[i,sorted_perm]

print(f"topk_block_indices={topk_block_indices}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
